In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas import DataFrame

# covid19.json 파일 읽어오기

In [2]:
# covid19.json 파일 읽어오기
with open('covid19.json', 'r') as file:
    covid = json.load(file)

# DataFrame 생성

In [3]:
covid_item = covid['response']['body']['items']['item']
covid_df = DataFrame(covid_item)
covid_df.head()

,accExamCnt,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accDefRate
0,21518073,2022-01-31 09:22:37.347,6755,845688,775,20220131,00:00,2022-02-07 09:06:04.861,NaN
1,21448119,2022-01-30 09:13:00.269,6732,828611,774,20220130,00:00,2022-02-07 09:05:54.937,NaN
2,21377393,2022-01-29 08:46:05.752,6712,811089,773,20220129,00:00,2022-02-04 08:59:02.84,NaN
3,21269304,2022-01-28 09:04:52.588,6678,793576,772,20220128,00:00,2022-02-04 08:58:55.443,NaN
4,21196354,2022-01-27 09:04:10.687,6654,777483,771,20220127,00:00,2022-02-02 09:05:52.264,NaN


# 데이터 분석

In [4]:
# 행과 열 정보
covid_df.shape

(742, 9)

In [5]:
# 전체 정보 : dtype
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   accExamCnt  742 non-null    object
 1   createDt    742 non-null    object
 2   deathCnt    742 non-null    object
 3   decideCnt   742 non-null    object
 4   seq         742 non-null    object
 5   stateDt     742 non-null    object
 6   stateTime   742 non-null    object
 7   updateDt    741 non-null    object
 8   accDefRate  673 non-null    object
dtypes: object(9)
memory usage: 52.3+ KB


In [6]:
# 결측치 개수
covid_df.isna().sum()

accExamCnt     0
createDt       0
deathCnt       0
decideCnt      0
seq            0
stateDt        0
stateTime      0
updateDt       1
accDefRate    69
dtype: int64

# 전처리

## 필요 없는 컬럼 삭제

In [7]:
# 필요 없는 컬럼 삭제
covid_df.drop(columns=['createDt','seq', 'stateTime', 'updateDt', 'accDefRate'])

,accExamCnt,deathCnt,decideCnt,stateDt
0,21518073,6755,845688,20220131
1,21448119,6732,828611,20220130
2,21377393,6712,811089,20220129
3,21269304,6678,793576,20220128
4,21196354,6654,777483,20220127
...,...,...,...,...
737,29,0,2,20200124
738,25,0,1,20200123
739,21,0,1,20200122
740,15,0,1,20200121


In [8]:
# 필요 없는 컬럼 삭제
covid_df.drop(columns=['createDt','seq', 'stateTime', 'updateDt', 'accDefRate'], inplace=True)

## 컬럼명 한글로 변경

In [9]:
# 컬럼명 한글로 변경
covid_df.columns = ['누적의심신고검사자', '사망자수', '확진자수', '기준일']
covid_df.columns

Index(['누적의심신고검사자', '사망자수', '확진자수', '기준일'], dtype='object')

## 컬럼 Dtype 변경

### 기준일 : object -> datetime
- date타입이 없기 때문에 datetime으로 변경
- datetime의 Dtype은 datetime64[ns]로 고정

In [10]:
# 방법 1
pd.to_datetime(covid_df.기준일)

0     2022-01-31
1     2022-01-30
2     2022-01-29
3     2022-01-28
4     2022-01-27
         ...    
737   2020-01-24
738   2020-01-23
739   2020-01-22
740   2020-01-21
741   2020-01-20
Name: 기준일, Length: 742, dtype: datetime64[ns]

In [11]:
# 방법 2
covid_df.기준일.astype('datetime64[ns]')

0     2022-01-31
1     2022-01-30
2     2022-01-29
3     2022-01-28
4     2022-01-27
         ...    
737   2020-01-24
738   2020-01-23
739   2020-01-22
740   2020-01-21
741   2020-01-20
Name: 기준일, Length: 742, dtype: datetime64[ns]

### 누적의심신고검사자수, 사망자수, 확진자수 : object -> int

In [12]:
covid_df.기준일 = covid_df.기준일.astype('datetime64[ns]')
covid_df = covid_df.astype({'누적의심신고검사자' : int, '사망자수' : int , '확진자수' : int})
covid_df.dtypes

누적의심신고검사자             int32
사망자수                  int32
확진자수                  int32
기준일          datetime64[ns]
dtype: object

## 컬럼 순서 변경

In [13]:
covid_df = covid_df[['기준일', '확진자수', '사망자수', '누적의심신고검사자']]
covid_df[:2]

,기준일,확진자수,사망자수,누적의심신고검사자
0,2022-01-31,845688,6755,21518073
1,2022-01-30,828611,6732,21448119


## 기준일 기준으로 오름차순 정렬

In [14]:
covid_df.sort_values(by='기준일', inplace=True)
covid_df[:2]

,기준일,확진자수,사망자수,누적의심신고검사자
741,2020-01-20,1,0,4
740,2020-01-21,1,0,15


## 인덱스 재설정
- 기존 인덱스 삭제, 원본 적용

In [15]:
covid_df.reset_index(drop=True, inplace=True)
covid_df[:2]

,기준일,확진자수,사망자수,누적의심신고검사자
0,2020-01-20,1,0,4
1,2020-01-21,1,0,15


## 컬럼명 변경
- 확진자수 -> 누적확진자수
- 사망자수 -> 누적사망자수

In [16]:
covid_df.rename(columns={'확진자수' : '누적확진자수', '사망자수' : '누적사망자수', '누적의심신고검사자' : '누적의심신고검사자수'}, inplace=True)
covid_df.columns

Index(['기준일', '누적확진자수', '누적사망자수', '누적의심신고검사자수'], dtype='object')

## 당일 확장자 수, 사망자 수 컬럼 추가

### 당일 확장자 수

In [17]:
# 확진자수 컬럼 추가
covid_df.insert(loc=1, column='확진자수', value=0)

In [18]:
covid_df.확진자수 = covid_df.누적확진자수 - covid_df.누적확진자수.shift(fill_value=0)
covid_df.확진자수

0          1
1          0
2          0
3          0
4          1
       ...  
737    14514
738    16093
739    17513
740    17522
741    17077
Name: 확진자수, Length: 742, dtype: int32

### 당일 사망자 수

In [19]:
# 사망자수 컬럼 추가
covid_df.insert(loc=3, column='사망자수', value=0)
covid_df[:2]

,기준일,확진자수,누적확진자수,사망자수,누적사망자수,누적의심신고검사자수
0,2020-01-20,1,1,0,0,4
1,2020-01-21,0,1,0,0,15


In [20]:
covid_df.사망자수 = covid_df.누적사망자수 - covid_df.누적사망자수.shift(fill_value=0)
covid_df

,기준일,확진자수,누적확진자수,사망자수,누적사망자수,누적의심신고검사자수
0,2020-01-20,1,1,0,0,4
1,2020-01-21,0,1,0,0,15
2,2020-01-22,0,1,0,0,21
3,2020-01-23,0,1,0,0,25
4,2020-01-24,1,2,0,0,29
...,...,...,...,...,...,...
737,2022-01-27,14514,777483,34,6654,21196354
738,2022-01-28,16093,793576,24,6678,21269304
739,2022-01-29,17513,811089,34,6712,21377393
740,2022-01-30,17522,828611,20,6732,21448119


# csv파일로 저장

In [21]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   기준일         742 non-null    datetime64[ns]
 1   확진자수        742 non-null    int32         
 2   누적확진자수      742 non-null    int32         
 3   사망자수        742 non-null    int32         
 4   누적사망자수      742 non-null    int32         
 5   누적의심신고검사자수  742 non-null    int32         
dtypes: datetime64[ns](1), int32(5)
memory usage: 20.4 KB


In [22]:
covid_df.to_csv('corona_table.csv', encoding='utf-8', index=False)